In [169]:
import pandas as pd

merged_data = pd.read_csv('data_simple.csv', encoding = "ISO-8859-1", low_memory=False, index_col=0)

In [170]:
merged_data.describe()

,ADD_DATE,ARREST_DATE,BAR_ADMISSION,CHARGE_CLASS,DFDN_SEQ_NBR,DOB,POLICE_RPT_DATE,POLICE_RPT_DAYS,SADA_DOB,SCREENING_DAYS,SCREENING_DISP_CODE,SCREENING_DISP_DATE,SYS_NBR
count,2.802900e+05,2.802900e+05,2.494070e+05,275692.000000,280294.000000,2.802940e+05,2.802940e+05,280294.000000,2.438620e+05,280294.000000,244779.00000,2.802940e+05,2.802940e+05
mean,1.979663e+07,1.935621e+07,1.989125e+07,3.275510,1.097765,1.934283e+07,1.712800e+07,32.258093,1.960577e+07,15.128747,238.36247,1.742101e+07,9.417777e+07
std,1.689978e+06,3.360846e+06,6.791726e+04,0.872969,0.483895,2.580123e+06,6.946953e+06,66.161661,8.182868e+04,34.381894,10.18528,6.628696e+06,3.470239e+06
min,0.000000e+00,0.000000e+00,1.905061e+07,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,1.901050e+07,0.000000,17.00000,0.000000e+00,8.400002e+07
25%,1.991063e+07,1.991031e+07,1.987101e+07,3.000000,1.000000,1.961013e+07,1.990082e+07,3.000000,1.960000e+07,0.000000,230.00000,1.990061e+07,9.110910e+07
50%,1.994090e+07,1.994053e+07,1.990100e+07,3.000000,1.000000,1.970103e+07,1.994052e+07,13.000000,1.963000e+07,5.000000,240.00000,1.994022e+07,9.417507e+07
75%,1.997071e+07,1.997052e+07,1.992102e+07,4.000000,1.000000,1.977082e+07,1.997062e+07,34.000000,1.966000e+07,16.000000,240.00000,1.997052e+07,9.724202e+07
max,1.999112e+07,1.999112e+07,1.999042e+07,8.000000,27.000000,1.999090e+07,1.999112e+07,999.000000,1.974000e+07,997.000000,560.00000,1.999112e+07,9.930916e+07


In [4]:
merged_data.columns

Index(['ADA_CODE', 'ADD_DATE', 'ARREST_CREDIT_CODE', 'ARREST_DATE',
       'BAR_ADMISSION', 'BOFI_NBR', 'CHARGE_CLASS', 'CHARGE_TYPE',
       'CRIMINAL_FLAG', 'DFDN_SEQ_NBR', 'DOB', 'FBI_NBR',
       'FINAL_DETENTION_FLAG', 'HABITUAL_OFFENDER_FLAG',
       'INITIAL_DETENTION_FLAG', 'JUVENILE_FLAG', 'LEAD_CHARGE_CODE', 'PARTY',
       'POLICE_RPT_DATE', 'POLICE_RPT_DAYS', 'RACE', 'SADA_DOB', 'SADA_RACE',
       'SADA_SEX', 'SCREENING_DAYS', 'SCREENING_DISP_CODE',
       'SCREENING_DISP_DATE', 'SEX', 'SYS_NBR'],
      dtype='object')

In [171]:
categorical = ['ARREST_CREDIT_CODE', 'CHARGE_CLASS', 'CHARGE_TYPE', 'LEAD_CHARGE_CODE',\
               'PARTY', 'RACE', 'SADA_RACE']

In [172]:
dict_counts = {}

for columnname in categorical:
    dict_counts[columnname] = merged_data[columnname].value_counts()

In [173]:
len(merged_data['ARREST_CREDIT_CODE'].unique())

103

In [174]:
merged_data[categorical].isnull().sum()

ARREST_CREDIT_CODE    10878
CHARGE_CLASS           4602
CHARGE_TYPE             581
LEAD_CHARGE_CODE        552
PARTY                 56195
RACE                   4821
SADA_RACE             38901
dtype: int64

In [175]:
merged_cat_filled = merged_data[categorical].fillna('NA') #fill NA's so that one-hot encoding creates dummy for "NA"

In [176]:
merged_cat_filled.isnull().sum()

ARREST_CREDIT_CODE    0
CHARGE_CLASS          0
CHARGE_TYPE           0
LEAD_CHARGE_CODE      0
PARTY                 0
RACE                  0
SADA_RACE             0
dtype: int64

### Remove invalid values

In [177]:
invalid_charge_codes = ['40:(979)296', 
               '14:(24)30(',
               '14:(24)67(',
               '5:606',
               '13:34',
               'F5:257',
               '14:(26)67(',
               '40:(979)1967',
               '4:664']

In [178]:
merged_cat_filled[merged_cat_filled['LEAD_CHARGE_CODE'].isin(invalid_charge_codes) == True]

,ARREST_CREDIT_CODE,CHARGE_CLASS,CHARGE_TYPE,LEAD_CHARGE_CODE,PARTY,RACE,SADA_RACE
UNIQUE_ID,,,,,,,
5207,NA,6,NA,5:606,D,NA,W
163070,05,NA,NA,40:(979)1967,D,N,W
196149,AL,4,AR,F5:257,R,NA,W
202726,05,NA,AR,13:34,D,N,B
203134,18,NA,AR,4:664,R,O,W
221631,56,NA,AR,40:(979)296,D,N,B
241333,07,NA,AR,14:(24)30(,D,W,W
241335,07,NA,AR,14:(24)30(,D,W,W
242808,22,2,AR,40:(979)296,D,W,T


In [179]:
merged_cat_filled['LEAD_CHARGE_CODE'].replace(invalid_charge_codes, 'NA', inplace=True)

In [180]:
merged_cat_filled['RACE'].replace('N', 'B', inplace=True)
#the codebook says that the value A should not be accepted, but this refers to "asian" elsewhere
#so I'm hesitant to throw out the A's
merged_cat_filled['RACE'].replace('O','A', inplace=True)

In [181]:
merged_cat_filled['ARREST_CREDIT_CODE'].replace('00', 'NA', inplace=True)

### Import table Code to verify interpretation of each category
#### ARCDCD in Code table: Arrest credit code
#### CGCD table: Lead charge code
#### CSCLC in Code table: Charge class

#### Lead Charge Code

In [182]:
CGCD = pd.read_stata("cgcd-cln.dta")
CGCD.to_csv('cgcd-cln.csv')

In [183]:
CGCD.head()

,charge_code,charge_cat,charge_class,charge_desc,expiration_date
0,CJP13(13),,6,IN NEED OF SUPERVISION,0
1,CJP13(14),,6,IN NEED OF CARE,0
2,06:668,GEN,4,OBTN LOAN/CRED. UNION BY FRAUD,0
3,12:315,GEN,4,MAIL FRAUD,0
4,12:501,GEN,4,CHARITY FAILURE TO FILE,0


In [184]:
Code = pd.read_stata("code-cln.dta")

Code.to_csv('code-cln.csv')

In [185]:
Code.head()

,code_type,code_code,short_desc,long_desc,add_date,add_time,add_user,change_date,change_time,change_user
0,ASDACD,ADOU,"DOUGLAS, A","DOUGLAS, ANDREW",0,0,,0,0,
1,ASDACD,AGOT,GOTTLIEB,"GOTTLIEB, ADRIAN",0,0,,0,0,
2,ASDACD,AGRE,"GREEN, A","GREEN, ALAN",0,0,,0,0,
3,ASDACD,ALAC,,"LACOUR, ADRIENNE",0,0,,0,0,
4,ASDACD,ALAM,"LAMBERT, A","LAMBERT, ANNE",0,0,,0,0,


In [186]:
merged_cat_filled = pd.merge(merged_cat_filled, CGCD[['charge_code', 'charge_class', 'charge_desc']], \
                             left_on=['LEAD_CHARGE_CODE'], right_on=['charge_code'], how='left')

In [188]:
len(merged_cat_filled.loc[merged_cat_filled['charge_code'].isnull() == True,'LEAD_CHARGE_CODE'].unique())

93

In [189]:
len(merged_cat_filled.loc[merged_cat_filled['charge_code'].isnull() == False,'LEAD_CHARGE_CODE'].unique())

846

93 lead charge codes that can NOT be connected to the lookup table of charge_codes.
846 that CAN be connected to the lookup table of charge codes.

In [216]:
unmatched = merged_cat_filled.loc[merged_cat_filled['charge_code'].isnull() == True,'LEAD_CHARGE_CODE'].unique()

a = pd.DataFrame(unmatched)

#### Instances where Charge Class of Lead Charge Code Does Not Match Charge Class from AREG table

In [214]:
non_null_cc = merged_cat_filled.loc[(merged_cat_filled['CHARGE_CLASS'] != 'NA') \
                                    & (merged_cat_filled['charge_class'].isnull() == False)\
                                    & (merged_cat_filled['charge_class'] != ''), ]

In [215]:
non_null_cc.loc[non_null_cc['CHARGE_CLASS'].astype(int).astype(str) != non_null_cc['charge_class'].astype(str),]

,ARREST_CREDIT_CODE,CHARGE_CLASS,CHARGE_TYPE,LEAD_CHARGE_CODE,PARTY,RACE,SADA_RACE,charge_code,charge_class,charge_desc
26,72,2,AR,40:967 (C) (2),R,B,W,40:967 (C) (2),3,POSS SCHEDULE 2 OTHER
72,72,3,AR,14:110 (A),R,B,W,14:110 (A),4,SIMPLE ESCAPE
197,31,3,AR,14:108,R,B,W,14:108,4,RESISTING AN OFFICER
242,05,2,AR,40:967 (C) (2),R,B,W,40:967 (C) (2),3,POSS SCHEDULE 2 OTHER
323,94,4,IF,40:969(C),NA,NA,NA,40:969(C),3,POSS SCHEDULE 4
372,NA,3,IF,14:110 (A),NA,B,NA,14:110 (A),4,SIMPLE ESCAPE
373,06,2,AR,40:967 (C) (2),R,B,W,40:967 (C) (2),3,POSS SCHEDULE 2 OTHER
701,22,2,AR,40:967 (C) (2),NA,B,NA,40:967 (C) (2),3,POSS SCHEDULE 2 OTHER
713,01,4,IF,40:969(C),NA,NA,NA,40:969(C),3,POSS SCHEDULE 4
723,22,4,AR,40:969(C),NA,W,W,40:969(C),3,POSS SCHEDULE 4


3,017 instances where the charge class associated with the lead_charge_code in the lookup table isn't the same as the charge class in areg

#### Arrest Credit Code

In [221]:
ARCDCD = Code.loc[Code['code_type'] == 'ARCDCD',]

In [224]:
merged_cat_filled = pd.merge(merged_cat_filled, ARCDCD[['code_code', 'long_desc']], \
                             left_on=['ARREST_CREDIT_CODE'], right_on=['code_code'], how='left')

In [226]:
merged_cat_filled.loc[merged_cat_filled['code_code'].isnull() == True,'ARREST_CREDIT_CODE'].unique()

array(['NA'], dtype=object)

#### The only arrest credit codes that don't align with the lookup table is "NA", so it seems there aren't any further data quality issues with this column

#### Charge Class (The Charge Class on AREG table)

In [237]:
CSCLCD = Code.loc[Code['code_type'] == 'CSCLCD',]

In [238]:
CSCLCD

,code_type,code_code,short_desc,long_desc,add_date,add_time,add_user,change_date,change_time,change_user
884,CSCLCD,1,CLASS 1,CAPITAL,19920802,12000000,MAP,19920802,12000000,MAP
885,CSCLCD,2,CLASS 2,"FELONY, HARD LABOR",19920802,12000000,MAP,19920802,12000000,MAP
886,CSCLCD,3,CLASS 3,"FELONY, OPTIONAL",19920802,12000000,MAP,19920802,12000000,MAP
887,CSCLCD,4,CLASS 4,MISDEMEANOR,19920802,12000000,MAP,19920802,12000000,MAP
903,CSCLCD,,,CASE CLASS CODES,0,0,,0,0,
1658,CSCLCD,5,CLASS 5,ASSET FORFEITURE,19931229,12000000,RBW,19960702,15000000,DPS
1659,CSCLCD,6,CLASS 6,JUVENILE STATUS,19931229,12000000,RBW,19960701,15000000,DPS
1664,CSCLCD,7,CLASS 7,POLICE STATUS,19940301,12000000,RBW,19960702,15000000,DPS
2116,CSCLCD,8,CLASS 8,MULTIPLE CLASS CHARGES IN CASE,1141998,0,SMR,0,0,


In [239]:
merged_cat_filled['CHARGE_CLASS'].unique()

array([3.0, 4.0, 2.0, 'NA', 1.0, 6.0, 8.0, 5.0, 7.0], dtype=object)

#### All the values in the merged file could be mapped to something in CSCLCD, so no further data quality issues with this column

### One Hot Encoding

In [90]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [114]:
'''
create features using one hot encoding
'''

# first convert into integer values. one hot enconding only takes int input

l_enc = LabelEncoder()

col1 = l_enc.fit_transform(merged_cat_filled['CHARGE_CLASS'].astype(str))
col2 = l_enc.fit_transform(merged_cat_filled['CHARGE_TYPE'])
col3 = l_enc.fit_transform(merged_cat_filled['LEAD_CHARGE_CODE'])
col4 = l_enc.fit_transform(merged_cat_filled['PARTY'])
col5 = l_enc.fit_transform(merged_cat_filled['RACE'])
col6 = l_enc.fit_transform(merged_cat_filled['SADA_RACE'])
col7 = l_enc.fit_transform(merged_cat_filled['ARREST_CREDIT_CODE'])

X = np.column_stack((col1,col2,col3,col4,col5,col6,col7))

#one hot encoding
enc = OneHotEncoder()
X_enc = enc.fit_transform(X)